# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
#data_path = 'data/'
df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [5]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因? 

1. df[object_features].mean()無法取平均。
2. df[object_features].max()可以執行。輸入的對象是Object，看字串比大小，由字串中第一個字母的ascii值比大小。
3. df['Cabin'].max() 和 df['Embarked'].max()會產生錯誤，因為浮點數和字串無法比大小。


# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類? 
  所以三大類特徵中，哪一大類處理起來應該最複雜?
1. bytes資料類型。bytes資料類型應該可歸在Object資料類型。
2. Object資料類型。Object資料類型處理起來應該是最複雜的，通常需要進行前置處理(ex: Label encoding或One Hot encoding等等)。

In [6]:
# 例 : 整數 (int) 特徵 取平均 (mean)
df[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [7]:
# 例 : 整數 (int) 特徵 取最大值 (max)
df[int_features].max()

Pclass    3
SibSp     8
Parch     9
dtype: int64

In [8]:
# 例 : 整數 (int) 特徵 取不同值的個數 (nunique)
df[int_features].nunique()

Pclass    3
SibSp     7
Parch     8
dtype: int64

In [9]:
# 例 : 浮點數 (float)特徵 取平均 (mean)
df[float_features].mean()

Age     29.881138
Fare    33.295479
dtype: float64

In [10]:
# 例 : 浮點數 (float)特徵 取最大值 (max)
df[float_features].max()

Age      80.0000
Fare    512.3292
dtype: float64

In [11]:
# 例 : 浮點數 (float)特徵 取不同值的個數 (nunique)
df[float_features].nunique()

Age      98
Fare    281
dtype: int64

In [12]:
# 例 : object 特徵 取平均 (mean)
df[object_features].mean()

Series([], dtype: float64)

In [13]:
# 例 : object 特徵 取最大值 (max)
df[object_features].max()

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

In [14]:
# 例 : object 特徵取不同值的個數 (nunique)
df[object_features].nunique()

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64

In [15]:
df['Cabin'].max()

TypeError: '>=' not supported between instances of 'float' and 'str'

In [16]:
df['Embarked'].max()

TypeError: '>=' not supported between instances of 'str' and 'float'

In [17]:
# 因為浮點數和字串無法比大小。
for data in df['Cabin']:
    print(f'資料:{data},型別：{type(data)}')

資料:nan,型別：<class 'float'>
資料:C85,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:C123,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:E46,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:G6,型別：<class 'str'>
資料:C103,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:D56,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:A6,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:C23 C25 C27,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:B78,型別：<class 'str'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'float'>
資料:nan,型別：<class 'floa

In [18]:
# 因為浮點數和字串無法比大小。
for data in df['Embarked']:
    print(f'資料:{data},型別：{type(data)}')

資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:S,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:C,型別：<class 'str'>
資料:Q,型別：<class 'str'>
資料:S,型別：<c